In [1]:
from typing import Callable, Dict, List, Optional, Tuple, Union

import h5py
# не удалять! import hdf5plugin !
import hdf5plugin
import pandas as pd
import torch
from torch.utils.data import Dataset

---------------------------------------------------

In [2]:
meta_file = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/metadata.csv'
df_meta = pd.read_csv(meta_file, index_col='cell_id')

In [3]:
df_meta.head(10)

,day,donor,cell_type,technology
cell_id,,,,
e0dde41ed6f2,3,27678,MasP,citeseq
25b1de7f18f6,3,27678,MkP,citeseq
59e175749a4c,3,27678,MkP,citeseq
cc43f415f240,3,27678,NeuP,citeseq
cf6cb48a1aca,3,27678,HSC,citeseq
7d03cdc2150c,3,27678,EryP,citeseq
ed27b16f6b29,3,27678,NeuP,citeseq
20a5293b5a5f,3,27678,NeuP,citeseq
9c110ee995b5,3,27678,HSC,citeseq


In [15]:
ids = list(df_meta[df_meta['technology'] == 'citeseq'].index)
print(len(ids))
print(ids[:10])

119191
['e0dde41ed6f2', '25b1de7f18f6', '59e175749a4c', 'cc43f415f240', 'cf6cb48a1aca', '7d03cdc2150c', 'ed27b16f6b29', '20a5293b5a5f', '9c110ee995b5', '655fb0bf81df']


------------------------------------------------------------------------------

In [5]:
eval_ids = pd.read_csv('/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/evaluation_ids.csv', index_col='cell_id')

In [6]:
mx = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_multi_inputs.h5'
my = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_multi_targets.h5'
cx = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_cite_inputs.h5'
cy = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/train_cite_targets.h5'

In [29]:
test_my = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/test_multi_inputs.h5'
test_cy = '/home/mks/PycharmProjects/multimodal_single_cell_integration/dataset/test_cite_inputs.h5'

------------------------------------------------------------------------------

## Изучение структуры

In [8]:
mi_flow = h5py.File(mx, 'r')
col_names = list(mi_flow[list(mi_flow.keys())[0]])

In [9]:
col_names

['axis0', 'axis1', 'block0_items', 'block0_values']

In [12]:
mi_flow[list(mi_flow.keys())[0]]['axis0'].shape

(228942,)

In [24]:
mi_flow[list(mi_flow.keys())[0]]['axis0'][145]

b'chr10:100653097-100653634'

In [13]:
mi_flow[list(mi_flow.keys())[0]]['axis1'].shape

(105942,)

In [17]:
mi_flow[list(mi_flow.keys())[0]]['axis1'][0]

b'56390cf1b95e'

In [14]:
mi_flow[list(mi_flow.keys())[0]]['block0_items'].shape

(228942,)

In [23]:
mi_flow[list(mi_flow.keys())[0]]['block0_items'][145]

b'chr10:100653097-100653634'

In [15]:
mi_flow[list(mi_flow.keys())[0]]['block0_values'].shape

(105942, 228942)

In [19]:
mi_flow[list(mi_flow.keys())[0]]['block0_values'][0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [20]:
mi_flow[list(mi_flow.keys())[0]]['block0_values'][0].shape

(228942,)

Итак, файл состоит из **названия участка генома, уникального идентификатора клетки, и большой numpy матрицы** являющейся ATAC данными.
Строчек в numpy массиве столько же сколько уникальных id клеток, таким образом строчка в матрице является вектором фичей конкретной клетки. Мы подразумеваем что порядок совпадает и сделан без ошибок. Так же у нас есть название для каждого столбца в матрице, указывающее на позицию ATAC фичи, но он полностью совпадает со списком позиций. Информация об участке на днк нам сейчас безполезна. Однако ее тоже можно обрабатывать.

**dna_pos:** mi_flow[list(mi_flow.keys())[0]]['axis0'] (список строк в битовом виде)

**cell_id:** mi_flow[list(mi_flow.keys())[0]]['axis1'] (список строк в битовом виде)

**atac_array:** mi_flow[list(mi_flow.keys())[0]]['block0_values']

In [25]:
list(mi_flow.keys())

['train_multi_inputs']

--------------------------------------------------------

In [33]:
my_flow = h5py.File(my, 'r')
print(list(my_flow.keys()))

['train_multi_targets']


In [36]:
col_names = list(my_flow['train_multi_targets'])

In [37]:
col_names

['axis0', 'axis1', 'block0_items', 'block0_values']

In [38]:
my_flow['train_multi_targets']['block0_items'][0]

b'ENSG00000121410'

In [41]:
my_flow['train_multi_targets']['axis0'][0]

b'ENSG00000121410'

--------------------------------------------------------